# Python API

In [1]:
from dynaparse import DynamicConfiguration

Instantiate a new configuration object using schema and values we've already defined.

In [2]:
dc = DynamicConfiguration(schema_file="sample/schema_nested.json", values_file="sample/values_nested.json")

By default, the configuration flattens all schema and uses dot notation to represent nested structures. This allows us to use the arg parser seamlessly and still have nested parameters.

In [3]:
dc.get_values()

{'boolean_parameter_1': True,
 'categorical_parameter_1': 'option1',
 'float_parameter_1': 1.0,
 'list_parameter_1': [0, 1, 2],
 'loss.str_parameter_1': None,
 'loss.int_parameter_1': 1}

However, we can still expand the nested parameters if we want.

In [4]:
dc.get_values(expand=True)

{'boolean_parameter_1': True,
 'categorical_parameter_1': 'option1',
 'float_parameter_1': 1.0,
 'list_parameter_1': [0, 1, 2],
 'loss': {'str_parameter_1': None, 'int_parameter_1': 1}}

If you don't specify a schema, it is inferred (and can be subsequently saved to edit later)

In [5]:
dc = DynamicConfiguration(values_file="sample/nested_config_values.json")

/Users/mwharton/Code/dynaparse/dynaparse/dynamic_configuration.py:72: UserWarning: No schema file specified, inferring schema from 'sample/nested_config_values.json'
  warnings.warn(


In [6]:
dc.get_values()

{'model.type': 'boosted_tree',
 'model.parameters.max_depth': 5,
 'model.parameters.max_leaves': 31,
 'model.parameters.bagging_fraction': 0.5,
 'model.parameters.eta': 0.03,
 'model.parameters.lr': 0.01,
 'train.data.paths': ['data/00*.csv', 'data/01*.csv'],
 'train.data.augmentation.method': 'fillna',
 'train.data.augmentation.column': 'x',
 'train.data.augmentation.kwargs.value': -1,
 'eval.data.paths': ['data/05*.csv']}

# ArgParse/CLI

Even though we have nested parameters, we can still set them dynamically using dot notation. Here's what the help script looks like.

In [7]:
!python example.py --config_schema sample/schema_nested.json --config_values sample/values_nested.json --help

PARSED ARGUMENTS:
usage: example1 [-h] [--config_schema CONFIG_SCHEMA]
                [--config_values CONFIG_VALUES] [--randomize_config]
                --boolean_parameter_1 BOOLEAN_PARAMETER_1
                --categorical_parameter_1 CATEGORICAL_PARAMETER_1
                --float_parameter_1 FLOAT_PARAMETER_1 --list_parameter_1
                LIST_PARAMETER_1 [LIST_PARAMETER_1 ...] --loss.str_parameter_1
                LOSS.STR_PARAMETER_1 --loss.int_parameter_1
                LOSS.INT_PARAMETER_1 [--param1 PARAM1]

Do stuff for example 1

optional arguments:
  -h, --help            show this help message and exit
  --config_schema CONFIG_SCHEMA
                        Dynamic configuration schema file specifying variable
                        named arguments (default: None)
  --config_values CONFIG_VALUES
                        File specifying values following the schema in
                        'config_schema'. These will override command line args
                    

Using a separate script, a pickled version of the parsed arguments was created. Let's inspect it.

In [8]:
import pickle
with open("parsed_args.p", "rb") as fd:
    args = pickle.load(fd)

In [9]:
args

Namespace(boolean_parameter_1=True, categorical_parameter_1='option1', config_schema='sample/schema_nested.json', config_values='sample/values_nested.json', float_parameter_1=1.0, list_parameter_1=[0, 1, 2], loss=namespace(int_parameter_1=1, str_parameter_1=None), param1=10, randomize_config=False)

A nested namespace was created, so we can access the parameters as any other python object.

In [10]:
args.boolean_parameter_1

True

In [11]:
args.categorical_parameter_1

'option1'

In [12]:
args.loss

namespace(int_parameter_1=1, str_parameter_1=None)

In [13]:
args.loss.int_parameter_1

1

In [14]:
args.loss.str_parameter_1

Finally, let's see what the help looks like if we don't specify a schema file

In [15]:
!python example.py --config_values sample/nested_config_values.json --help

/Users/mwharton/Code/dynaparse/dynaparse/dynamic_configuration.py:72: UserWarning: No schema file specified, inferring schema from 'sample/nested_config_values.json'
  warnings.warn(
PARSED ARGUMENTS:
usage: example1 [-h] [--config_schema CONFIG_SCHEMA]
                [--config_values CONFIG_VALUES] [--randomize_config]
                --model.type MODEL.TYPE --model.parameters.max_depth
                MODEL.PARAMETERS.MAX_DEPTH --model.parameters.max_leaves
                MODEL.PARAMETERS.MAX_LEAVES
                --model.parameters.bagging_fraction
                MODEL.PARAMETERS.BAGGING_FRACTION --model.parameters.eta
                MODEL.PARAMETERS.ETA --model.parameters.lr MODEL.PARAMETERS.LR
                --train.data.paths TRAIN.DATA.PATHS [TRAIN.DATA.PATHS ...]
                --train.data.augmentation.method
                TRAIN.DATA.AUGMENTATION.METHOD
                --train.data.augmentation.column
                TRAIN.DATA.AUGMENTATION.COLUMN
                --tr